# Assignment 9: Continuous-time Markov chains and Poisson processes

## EEB 463

## Introduction

This week, we will be exploring the behaviour of continuous-time Markov chains to get a stronger intuition for their behaviour and properties.

## Q1. Simulating DNA substitutions

For this question, you will construct a Q matrix for a CTMC model describing transitions between DNA bases, Adenine, Guanine, Cytosine, and Thymine. Remember that nucleotide bases fall into one of two types: purines (A and G) and pyrimidines (C and T). We will model substitutions by assuming two rates exist. One rate, which we will refer to as $r_i$, will describe the instantaneous rate of transitions, i.e., substitution between bases of the same type (i.e., purine -> purine or pyrimidine -> pyrimidine). The other rate, which we will refer to as $r_v$, will capture the rate of transversions, or substitutions across types.

### Q1a.

Please complete the function below to return a Q matrix, given imputed values for $r_i$ and $r_v$. Remember that each row gives the rate of moving from one base to another, based on column order. Use the order below. The column order should match the row order.

row 0: A 

row 1: G

row 2: C

row 3: T

_From this point forward, we will also map each of these integers to the corresponding base. So, under the CTMC, state 0 will refer to adenine, state 1 to guanine, etc._

In [ ]:
# order: 0: A, 1: G, 2: C, 3: T
def set_Q(ri,rv):
    Q = np.array(
        [ 
        [-(ri + (rv * 2.)), ri, rv, rv], 
        [ri, -(ri + (rv * 2.)), rv, rv],
        [rv, rv, -(ri + (rv * 2.)), ri],
        [rv, rv, ri, -(ri + (rv * 2.))]
        ])
    return Q

### Q1b.

Create another function that simulates a realization of the CTMC given _Q_ and _dt_. In lecture, we used an example of a CTMC for a two-state (binary) character. Now, we are simulating under a four-state character. For a binary character, we determined if a change by drawing a random number and comparing it to the probability of changing state, based on the off-diagonal element in the row of the _P_ matrix corresponding to the current state. Here, if a change occurs, we could change into one of 3 alternatives. You may find it simplest to use `random.choices()`, which will select a random element from a list using a set of weights (which can be imputed using the `weights` parameter). The weight of moving to a character state is equal to its probability given _P_.

In [ ]:
import numpy as np
import random
from scipy.linalg import expm

def sim_DNA(Q, time_window, dt = 0.1):
    char_state = 0
    states = [char_state]
    curtime = 0.
    times = [curtime]
    while curtime < time_window:
        curtime += dt
        P = expm(Q*dt)
        cur_row = P[char_state]  # we'll pull out the row corresponding to our current state
        prob_change = 1. - cur_row[char_state]  # find probability of change by pulling index opposite of char_state
        
        newstate = random.choices([i for i in range(4)], weights = cur_row, k = 1)[0]
        char_state = newstate
        states.append(char_state)
        times.append(curtime)
    return times, states

### Q1c.

Run the function using a time window of 20, $r_i$ = 0.2 and $r_v$ = 0.1. Plot the results like we did in class to visualize all the substitutions over the time window.

In [ ]:
import matplotlib.pyplot as plt

ri = 0.2
rv = 0.1
times,states = sim_DNA(set_Q(ri,rv),20.)
plt.plot(times,states)
plt.show()

## Q2. Inference of the CTMC

### Q2a.

Complete the function below to compute the log-likelihood of a time series of discrete states based on the model implemented above.

In [ ]:
def calc_nuc_loglike(times, data, ri, rv):
    ll = 0.0
    for i in range(1,len(times)):
        cur_state = data[i]
        last_state = data[i-1]
        dt = times[i] - times[i-1]
        Q = set_Q(ri,rv)
        P = expm(Q*dt)
        like = P[last_state][cur_state]
        stepll = math.log(like)
        ll += stepll
    return ll

### Q2b.

Use code similar to that used in lecture to use this likelihood function to visualise the 2-dimensional likelihood surface of our rate parameters, $r_i$ and $r_v$. Make sure to update the ranges of parameters as well as the levels plotted, since the likelihoods will likely be scaled differently from the in-class example. After computing the likelihoods for all the combinations of parameter values, you might use the `min()` and `max()` functions to find the upper and lower thresholds of the computed likelihoods in order to determine an appropriate range of levels.

In [ ]:
import math

X = np.linspace(.01, .4, 40)
Y = np.linspace(.01, .4, 40)
Xgrid, Ygrid = np.meshgrid(X, Y)
z = []

for yi in Y: 
    y_ll = []
    for xi in X:
        ll = calc_nuc_loglike(times,states,xi,yi)
        y_ll.append(ll)
        
    z.append(y_ll)
    
levels = np.linspace(min(min(z)),max(max(z)),7) 
fig, ax = plt.subplots()
CS = ax.contour(Xgrid,Ygrid,z,levels=levels)
ax.clabel(CS, inline=True, fontsize=10)
plt.xlabel("ri")
plt.ylabel("rv")
plt.show()

### Q2c.

Comment on the likelihood surface produced. Are the parameters more or less identifiable than the two-state model inferred in class? Explain any differences you see. 

**PROVIDE ANSWER HERE**

## Q3. **Stationary Distribution**

In you previous assignment you used the tail end of the MCMC to get what is known as its 'stationary distribution'.  In this question, we will gain intuition about what a stationary distribution and how to compute it in python for the CTCM of transitions and transversions. 

The stationary distribution ${\pi} = (\pi_0,\pi_1,\pi_2,\pi_3)$ of the CTMC is the "equilibrium" point of the CTMC, i.e., once the probability of being in state $i$ becomes $\pi_i$ for all $i$, the probability distribution of being in a given state stops changing. 

### **Q3a** 

The stationary distribution ${\pi}$ of a CTMC with transition matrix *Q* is given by solving the following linear equations 

<center> $\mathbf{Q}^T\pi^T = 0$ <center>
<center> $\displaystyle \sum_{i} \pi_i = 1$ <center>
    
To do this we will create a matrix $\mathbf{A} = \begin{bmatrix} \mathbf{Q} \\ \mathbf{1} \end{bmatrix} $ where $\mathbf{1} = [1,1,1,1]$. and $\mathbf{b} = \begin{bmatrix} 0 \\ 0 \\ 0 \\ 0 \\ 1 \end{bmatrix}$. Then we can obtain the stationary distribution by solving $\mathbf{A}\pi=\mathbf{b}$.
    
You can use `np.linalg.lstsq()` to do this. The first entry (which will be a list of 4 values) of the output is the stationary distribution.  
 

In [ ]:
A = np.append(Q, np.transpose([[1,1,1,1]]), axis=1)
A = np.transpose(A)
b= np.array([0, 0, 0, 0, 1])
x, residuals, rank, s = np.linalg.lstsq(A, b, rcond=None)
print(x)

### **Q3b** 
What is the probability of being in each state under the stationary distribution? Why does this make sense given the rates of transitions and transversions?

WRITE ANSWER HERE

### Q3c 

From timeseries data of states from Q1c, make a histogram of the last 100 values. Now rerun `sim_DNA` with same values but for a longer time window (say 1000 units). Now plot a histogram for the timeseries data (which should have 10000 entries) excluding the first 1000 points. 

NOTE: For the histogram, use `bins = [-0.5,0.5,1.5,2.5,3.5]` and `density=True`.

In which of the two cases above is the plotted histogram closer to the stationary distribution computed in Q3a? Give two reasons why? HINT: You might want to plot the histogram for last 100 values for the second case.  

In [ ]:
ri = 0.2
rv = 0.1
times,states = sim_DNA(set_Q(ri,rv),20.)

plt.figure()
plt.hist(states[-100:], bins = [-0.5,0.5,1.5,2.5,3.5], density = True)

ri = 0.2
rv = 0.1
times,states = sim_DNA(set_Q(ri,rv),1000.)

plt.figure()
plt.hist(states[1000:], bins = [-0.5,0.5,1.5,2.5,3.5], density = True)

### Q3d.

Suppose you had a different Q matrix for the same problem, given by Q = $\begin{bmatrix} -0.4 & 0.2 & 0.1 & 0.1 \\ 0.2 & -0.4 & 0.1 & 0.1 \\ 0.2 & 0.2 & -0.8 & 0.4 \\ 0.2 & 0.2 & 0.4 & -0.8  \end{bmatrix}$ 

Given this new Q matrix answer the following questions 

**Q3d(i)** Compare the new Q matrix to one from previous questions. What is the difference mathematically (like what entries are different)? 

ANSWER HERE 

**Q3d(ii)** Give a biological explanation for this new Q matrix, i.e., what does this tell us about the rates of transitions and transversions in purines vs in pyrimidines?

ANSWER HERE

**Q3d(iii)** Without doing the computations or running any code, guess what the stationary distribution will look like under this new Q matrix?

ANSWER HERE